# Paper1: Divide to Adapt Review:
* [link](https://openreview.net/pdf?id=hVrXUps3LFA)
* [code](https://github.com/xyupeng) 

* main algorithm:
    1. divide the target domain into 2 domains: easy and hard. Using the distribution of the loss
    2. Even though the number of mislabed samples in the easy domain is quit high, mislabeled samples are inevitable
    3. They use Mutually distilled twin network to generate two domains (still not quite clear)

* Competiting methods with references:
    1. sample selection: [Unsupervised Domain Adaptation of Black-Box Source Models](https://arxiv.org/abs/2101.02839)
    2. single destillation: [DINE: Domain Adaptation from Single and Multiple Black-box Predictors](https://arxiv.org/abs/2104.01539)

* disadvantages of competing methods: 
    1. confirmation bias definied in general [as](https://developers.google.com/machine-learning/crash-course/fairness/types-of-bias). In case of ***DA***, it is the accumlation of errors from uncorrect labels which makes the model overfit to noisy labels and enable to learn actual domain invariant features. For [details](https://arxiv.org/abs/1908.02983)
    2. the selection of clean samples limits the model's capabilities by not laveraging the data to its full potenial
    3. theoretical justifications (well that's probably (definitely) out of my reach for the moment)

* The domain division is not absolutely clear yet. Here is what I can say about it: 
    1. Assuming $K$ classes, then we can use the loss $$L = -\sum_{c=1}^{K} y^{i}_c \cdot log(p^i_c) $$ where $y^i_c$ is the indicator that sample $i$ belongs to class $c$ and $p^i_c$ is the probability of sample $i$ belong to class $c$ (using softmax). $L$ ranges from $0$ to $1$
    2. Assuming we make make a histogram where $x$ axis is the loss and $y$ is the number of samples that achieves that loss (well in bins but u got what I mean). we can get something like this:![distribution](../photos/GMM.png) 
    3. They extract some probabilities using ***GMM*** which gives both the division and its distribution (Which is further used in the labeling)
* No clue about the twin network as I don't have background knowledge yet.

* Direct Readings: 
    1. Domain Distillation (see the reference then ask Khan)
    2. Self supervised Learning 
    3. Twin Network
    4. Warm up for networks 

* Indirect reads:
    1. GMM
    2. some other mathematical ideas such as KL divergence, A-distance

# Paper2: DINE: DIstill and finetuNE
* [link](https://arxiv.org/pdf/2104.01539.pdf)
* [code](https://github.com/tim-learn/DINE/)

* main ideas:
    1. They add a linear layer and train its weight using Label Smoothing. Now, they have a source model better suited for the DA task
    2. Apply an adaptive Knowledge distillation procedure with adaptive Label Smoothing (Since the source model predictions for the target domain are noisy)
    3. add structural regularization (that I don't understand yet)
    4. fine tune the predictions.

* main merit:
    1. mainly using the predictor as a white box: having access to the inner layers of the model

* competings references:
    1. [UDA without source data](https://openaccess.thecvf.com/content_CVPR_2020/papers/Li_Model_Adaptation_Unsupervised_Domain_Adaptation_Without_Source_Data_CVPR_2020_paper.pdf). uses the white box predictor 
    2. [Hypothesis Transfer and Labeling Transfer](https://arxiv.org/pdf/2012.07297.pdf). Black box predictor


* Direct Readings: 
    1. [Vanilla Knowledge distillation](https://arxiv.org/pdf/1503.02531.pdf) by Google Research team
    2. [Label Smoothing]()


# Paper 3: Vanilla Knowledge Distillation 
* [link](https://arxiv.org/pdf/1503.02531.pdf)
* no code link in the paper

* main ideas:
    1. The main idea is a random variable with high entropy is quite informative. In other words, if we have an instance $x_i$, associated with probabilities $p_A = 10 ^ {-6}$, $p_B = 8 * 10 ^ {-5}$ and $p_C = 10 ^ {-2}$, if similar distributions are observed across a large number of instances we can say that $A$ and $B$ are quite similar. 
    2. Basically, soft labels incorporate much more knowledge than hard labels.
    3. Cumbersome model (CM) and distilled Model (DM). Train the model on some large training data. leave a holdout dataset generally referred to as the ***transfer dataset***. Minimize entropy between the logits (probabilities are too low to affect the entropy loss) of CM and DM
    4. Having some of transfer data labeled (or all of it) with CM's predictions. The loss function can be updated by adding a term where the DM should predict correctly.
    5. The mathematical details are not shared, so I can't tell for sure how the process work.
    6. The softmax function is defined as: $$p_i = \frac{exp(\frac{z_i}{T})}{\sum_j exp(\frac{z_j}{T})} $$
    7. The value $T$ is referred to as the temperature. The larger the value of $T$, the softer, more uniform the distribution will get.
    8. if DM is relatively large, high values of $T$ should be used. For relatively smaller DMs (less neuron units, less layers), use smaller $T$
    9. $T$ is used in the softmax predictions of the DM model on the transfer dataset


* funny enough (I am still not used to how research papers work), this [FitNet](https://arxiv.org/pdf/1412.6550.pdf) explains the vanilla Knowledge Distillation:
    1. So when the labels are not known, the student is trained to minimize $$ L(P^{\tau}_{T}, P^{\tau}_{S})$$ where $P^{\tau}$ is the output of the softmax with temperature $T$
    2. when some of the labeled data is known, the loss can be modified as follows: $$ L(y_{true}, P^{\tau}_{S}) + \lambda \cdot L(P^{\tau}_{T}, P^{\tau}_{S})$$


The paper share a number of ideas concerning distilling ensemble of models, but it does not serve the topic of my research (at least I don't see how, yet).
Reading Smooth labeling is a must.


# Paper 4: Label Smoothing: Rethinking the inception architecture
Funny enough, the paper is about the inception architecture, but it is used as the main reference of label smoothing. Label smoothing is recently used as a regularization mechanism.

* [link](https://www.cv-foundation.org/openaccess/content_cvpr_2016/papers/Szegedy_Rethinking_the_Inception_CVPR_2016_paper.pdf)
* The paper introduces interesting general principles to design powerful Neural Networks architectures. I didn't get most of them as I read in the few hours of the day
* The paper splendidly explains Label smoothing: 

* Mathematical break:
    1. let's denote $z_i$ the logits of the i-th class in a $K$ label classification problem.
    2. we have $p(i|x) = \frac{exp(z_i)}{\sum_{j=1}^{k} exp(z_j)}$
    3. Assuming we have a probability distribution for the labels denoted as $q(k|x)$, technically the probability of the example $x$ to have a label $k$, the loss function to minimize is: $$ l = - \sum_{i=1}^{K} \log(p(i)) \cdot  q(i)$$
    4. we have $$\frac{\delta l}{\delta z_i} = p(i) - q(i)$$ (easy to prove) which is bounded $\in [-1, 1]$
    5. minimizing $l$ is simply maximizing the log prediction for $P(i|x)$ (the correct label for x), which cannot achieved for finite $z_i$ but can be approached if $z_i$ is really large in comparison to other logits
    6. This introduces 2 major problems:
        * The model is unlikely to generalize as it lears to assign full probability to the correct label
        * It migh introduce issues with the gradients (the model is too confident in its predictions)
        * probably a deeper understanding of the mathematics of back propagation would help but that should be enough for no

    7. The solution is to use soft labels, generally defined as: $$ q^{'}(k|x) = (1 - \epsilon) \delta_{k,y} + \epsilon \cdot u(k)$$
    8. The research team defined $u(K)$ as the uniform distribution. meaning the correct label has $1 -\epsilon + \frac{\epsilon}{K}$ while uncorrect labels have a probabilty of $\frac{\epsilon}{K}$
    9. soft labeling serves as a powerful technique for regularization and handling uncorrect labels.


# Paper5: FiTNets: hint-based Knonwledge Destillation
* [link](https://arxiv.org/pdf/1412.6550.pdf)
* [code](https://github.com/adri-romsor/FitNets)

* The main idea:
    1. Using the hidden layers of the teacher model in the training
    2. Training the student model progressively, by training a selection of layers (they should be in the beginning of the network as later layers are more restrective
    and have less transferable knowledge)
    3. optimize each layer by optimizing the loss function: $L(W_{guided}, W_{r}) = \frac{1}{2} \cdot ||u_{t}(x, W_{hint}) - r\cdot(u_{s}(x, W_{guided}), W_r)||$  
    where $u_{t}$ is the output of the teacher's model up to that layer (with parameters $W_{hint}$). and $u_{s}$ is the student model output up to the guided layer. 
    4. it is important to keep in mind that the shape of the guided layer output  might be different from the one with the hint layer. Thus, the output should be modified with the regressor (with parameters $W_r$).
    5. The regressor can be chosen as a convolution neural network for efficienty (instead of a fully connected network which is likely to introduce signficant overhead)
    6. after assigning the new parameters for the different hidden layers in the student model, the student model is further trained with the Vanillay distillation loss function.
![algorithm's summary](../photos/FitNetKD.png)

# Paper 6: Knowledge Distillation
* [link](https://arxiv.org/pdf/1904.01866.pdf)
* [code](https://github.com/clovaai/overhaul-distillation)

* main ideas:
    1. introducing the 5 main design aspects of a Knowledge Destillation mechanism:
        * Teacher transform: a transformation that makes the teacher's features easy to transfer, learn. This is where most information is lost
        * Student transform: the transformation applied 
        * distillation distance: the function to be optimized during the student's model training 
        * distillation position: the positions in teacher and students model from which the outputs are extracted
    2. The authors make use of the RELU activation function as they extract input before RELU layer, neglecting the negative values (as it is reduced by the Relu function)
    3. They pass the values of a student by a marginal RELU: $$\sigma(x) = max(x, m)$$ where $m < 0$ 
    3. They introduce a modification of $L_2$ by considering the square difference between $S_i$ and $T_i$ only if $T_i \geq S_i$

Here is a couple of illustrations:
![algorithm's summary](../photos/KD_losses.png) 

The choice of $m$ for $\sigma$ in each distillation point, is the most interesting part. it uses some statistical ideas.

# Paper5: Knowledge Distillation Methods for Efficient Unsupervised Adaptation Across Multiple Domains
* [link](https://arxiv.org/pdf/2101.07308.pdf)
* I didn't find anything ground breaking (or probably I am just too stupid)
* for some reason they seem to retrain the 


# Paper 7: U Multi-Target DA through KD
* I can't sense any worth in this paper
* [link](https://arxiv.org/pdf/2007.07077v4.pdf)
* The 2 papers above are quite similar and seem to just combine the 2 words of UDA and KD for the sake of it, modifying the loss functions, order of certains operations but no particularly significant ground breaking 